<a href="https://colab.research.google.com/github/AS-AIGC/AS-AIGVTS/blob/main/colab_notebook_AS_AIGVTS_Transcript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Generated Multilingual Transcriptions for YouTube Video by Academia Sinica (AS-AIGVTS-Transcript)

## 中研院科普演講影音平台現況
- ✅ 知識寶庫，主題多元
- ✅ 大師雲集，內容精彩

- ⚠️ 偏好新上架或熱門影片
- ⚠️ 僅能從主題進行搜尋

- ⛔️ 沒有字幕，不夠友善
- ⛔️ 無法吸引國外人士閱聽
- ⛔️ 無法提供快速導覽
- ⛔️ 無法提供友善搜尋

## 我們的解法

- 使用 OpenAI Whisper 在地端進行語音辨識轉字幕檔
- 使用 Googletrans 進行多國語言翻譯
- *(用 ChatGPT API 為影片內容撰寫摘要)
- *(用 Googletrans 將摘要翻譯成多國語言版本)

## Demo

- 【生成式 AI】Diffusion Model 原理剖析 (1/4) (optional), by 李宏毅 [[原始連結](https://www.youtube.com/watch?v=ifCDXFdeaaM)] [[加字幕後連結](https://www.youtube.com/watch?v=-_FnWFL1LLk)]

## 註記

1. () 內的內容在本 Colab Notebook 中沒有提供，可參考 GitHub 上之 Python 版本程式 [[GitHub: AS-AIGC/AS-AIGVTS](https://github.com/AS-AIGC/AS-AIGVTS)]
2. 以下程式內容說明與註解，皆由 ChatGPT 產生，並經人工簡略編輯而成。


## Initialization

我們在這段程式碼中做了幾件事。首先，我們安裝了 pytube 庫，用於下載 YouTube 影片。其次，我們安裝了 pydub 庫，用來處理音訊檔。接著，我們安裝了 openai 和 openai-whisper 庫，來存取 OpenAI 的語言模型。然後，我們安裝了 pysrt 庫，將文字轉換成 srt 檔案格式。最後，我們安裝了 googletrans 庫，用於翻譯字幕。

In this block of code, we're doing several things. First, we're installing the pytube library, which allows us to download YouTube videos. Then, we're installing the pydub library to work with audio files. After that, we're installing both openai and openai-whisper libraries, for accessing OpenAI's language model. Next, we install the pysrt package to convert text into an srt file format. Lastly, we're installing the googletrans library to translate captions.

In [ ]:
# Install the Pytube library for downloading YouTube videos
!pip install -q --upgrade pytube

# Install the Pydub library for working with audio files
!pip install -q --upgrade pydub

# Install the OpenAI library for accessing OpenAI's GPT-3 language model
!pip install -q --upgrade openai

# Install the OpenAI-Whisper library for securely accessing OpenAI's GPT-3 language model
!pip install -q --upgrade openai-whisper

# Install the srt package for transforming the text to srt file format.
!pip install pysrt

# Install the googletrans for translate the caption
!pip install googletrans==3.1.0a0

在這段程式碼中，我們匯入了我們所需要的各種模組和庫。首先，我們匯入了 os 和 json 模組，用於操作系統和處理 JSON 資料。然後，我們匯入了 datetime，用於處理日期和時間。接著，我們匯入了 openai，以便使用 GPT 語言模型。再來，我們從 pytube 匯入 YouTube，用於下載 YouTube 影片，並從 pydub 匯入 AudioSegment 來處理音訊檔。然後，我們匯入了 whisper 和其 get_writer 函數，用於使用 whisper 語音識別系統。最後，我們從 pysrt 匯入 pysrt 來轉換文字成 srt 檔案格式，並從 googletrans 匯入 Translator 和 LANGUAGES，用於翻譯字幕。

In this block of code, we're importing various modules and libraries that we need. First, we're importing the os and json modules, for interacting with the system and handling JSON data. Then, we're importing datetime for dealing with dates and times. After that, we're importing openai so that we can use the GPT language model. Next, we import YouTube from pytube for downloading YouTube videos, and AudioSegment from pydub to work with audio files. Then, we import whisper and its get_writer function for working with the whisper voice recognition system. Finally, we import pysrt from pysrt to convert text into an srt file format and Translator and LANGUAGES from googletrans for translating captions.

In [ ]:
import os  # This module provides a way of using operating system dependent functionality.
import json  # This module provides a way for us to work with JSON data.
from datetime import datetime  # We're importing the datetime class from the datetime module. This will help us handle date and time related tasks.
import openai  # We're importing the OpenAI library so that we can use its GPT-3 language model.
from pytube import YouTube  # We're importing the YouTube class from the pytube library for downloading YouTube videos.
from pydub import AudioSegment  # We're importing the AudioSegment class from the pydub library for working with audio files.
import whisper  # We're importing the whisper library to work with the Whisper ASR API.
from whisper.utils import get_writer  # We're importing the get_writer function from the whisper.utils module.
import pysrt  # We're importing the pysrt library to handle SubRip (SRT) subtitle files.
from googletrans import Translator, LANGUAGES  # We're importing the Translator class and the LANGUAGES dictionary from the googletrans library for translating text.

## Defining Functions

在這段程式碼中，我們定義了一個函數 download_Youtube_video_audio_file，其目的是從 YouTube 下載影片的音訊檔。首先，我們印出要下載的 URL。然後，我們用提供的 URL 建立了一個 YouTube 物件。接著，我們找到影片的音訊軌道。最後，我們將音訊檔下載到指定的輸出資料夾，並使用提供的檔名。

In this block of code, we're defining a function download_Youtube_video_audio_file that's designed to download the audio file of a YouTube video. We first print out the URL that we're going to download. Then, we create a YouTube object with the provided URL. After that, we get the audio track of the video. Finally, we download the audio file to the specified output directory and with the provided filename.

In [ ]:
def download_Youtube_video_audio_file(url, output_directory, audio_filename):
  print(f"Download url: {url}")  # We're printing the URL that we're going to download.
  yt = YouTube(use_oauth=True, url=url)  # We're creating a YouTube object with the provided URL.
  # Get video's audio track
  audio_stream = yt.streams.filter(only_audio=True).first()  # We're getting the audio track of the video.
  audio_stream = yt.streams.get_audio_only()  # We're getting the audio track of the video.

  # Download it to Audio directory
  audio_stream.download(output_path=output_directory, filename=audio_filename)  # We're downloading the audio file to the specified output directory and with the provided filename.

在這段程式碼中，我們定義了一個函數 convert_audio_file_to_mp3_format，用來將音訊檔轉換成 MP3 格式。首先，我們載入音訊檔到一個 AudioSegment 物件。然後，我們將音訊檔轉換成 MP3 格式，並將它儲存到指定的路徑。

In this block of code, we're defining a function convert_audio_file_to_mp3_format to convert an audio file to MP3 format. We first load the audio file into an AudioSegment object. Then, we convert the audio file to MP3 format and save it to the specified path.

In [ ]:
def convert_audio_file_to_mp3_format(audio_filepath, export_path):
  # Load the audio file into an AudioSegment object
  print("Load the audio file")  # We're printing that we're about to load the audio file.
  audio_file = AudioSegment.from_file(audio_filepath)  # We're loading the audio file into an AudioSegment object.

  # Convert the audio file to MP3 format and save it to the mp3_format
  print("Convert the audio file to MP3 format\n")  # We're printing that we're about to convert the audio file to MP3 format.
  mp3_file = audio_file.export(export_path, format="mp3")  # We're converting the audio file to MP3 format and saving it to the specified path.

在這段程式碼中，我們定義了一個函數 slice_audio，用來將音訊檔切成一分鐘的片段。首先，我們獲取音訊檔的總長度（以秒為單位），並計算出總長度是多少分鐘。然後，我們將一分鐘轉換成毫秒。接著，我們設定開始和結束的時間戳記。如果開始的時間戳記等於總分鐘數，那麼結束的時間就是音訊的總長度，否則結束的時間就是下一分鐘。最後，我們將切割出來的音訊檔儲存成 MP3 格式。

In this block of code, we're defining a function slice_audio to slice an audio file into one-minute segments. We first get the total duration of the audio file in seconds and calculate how many minutes that is. Then, we convert one minute into milliseconds. After that, we set the start and end timestamp. If the start timestamp equals the total minutes, then the end time is the total duration of the audio, otherwise, the end time is the next minute. Finally, we export the sliced audio file as an MP3 format.

In [ ]:
def slice_audio(audio_file, filename, offset):
  # pydub does things in milliseconds
  audio_length = audio_file.duration_seconds  # We're getting the total duration of the audio file in seconds.
  minutes_duartion = int(audio_length // 60)  # We're calculating how many minutes the total duration is.

  one_minutes = 1 * 60 * 1000  # We're converting one minute into milliseconds.
  # Set the start and end timestamp
  start = offset * one_minutes  # We're setting the start timestamp.
  # The last part is less than one minute
  end = audio_length if start == minutes_duartion else (offset+1) * one_minutes  # We're setting the end timestamp.
  sliced_audio = audio_file[start:end]  # We're slicing the audio file from the start to the end timestamp.
  sliced_audio.export(filename, format="mp3")  # We're exporting the sliced audio file as an MP3 format.

在這段程式碼中，我們定義了一個函數 concatenate_srt_file，用來連接兩個 srt 字幕檔。首先，我們打開兩個 srt 字幕檔。然後，我們將切割出來的字幕檔的開始和結束時間戳記做位移，位移的數量是給定的分鐘數。接著，我們將切割出來的字幕檔的字幕索引做位移，位移的數量是主要字幕檔的長度。最後，我們將切割出來的字幕檔加到主要字幕檔後面，並儲存主要字幕檔。

In this block of code, we're defining a function concatenate_srt_file to concatenate two srt subtitle files. We first open the two srt files. Then, we shift the start and end timestamp of the sliced subtitle file by the given number of minutes. After that, we shift the subtitle index of the sliced subtitle file by the length of the main subtitle file. Finally, we append the sliced subtitle file to the main subtitle file and save the main subtitle file.

In [ ]:
def concatenate_srt_file(main, sliced_part, offset):
  # Open the srt file
  main_subtitles = pysrt.open(main)  # We're opening the main srt file.
  sliced_part_subtitles = pysrt.open(sliced_part)  # We're opening the sliced srt file.

  # Shift start and end timestamp of the sliced part subtitles 
  sliced_part_subtitles.shift(minutes=offset)  # We're shifting the start and end timestamp of the sliced subtitle file by the given number of minutes.

  # Shift subtiltes index of the sliced part subtitles
  main_subtitles_length = len(main_subtitles)  # We're getting the length of the main subtitle file.
  for subtitle in sliced_part_subtitles:  # For each subtitle in the sliced subtitle file...
    subtitle.index += main_subtitles_length  # ...we shift its index by the length of the main subtitle file...
    main_subtitles.append(subtitle)  # ...and then we append it to the main subtitle file.
  main_subtitles.save(main, encoding='utf-8')  # We're saving the main subtitle file.

在這段程式碼中，我們定義了一個函數 translate_srt_file_by_googletrans，用來利用 Google 翻譯服務將 srt 字幕檔翻譯成指定的語言。首先，我們建立一個翻譯器物件。然後，我們打開 srt 字幕檔，並對字幕檔中的每一個字幕進行翻譯。我們將每個字幕的文字翻譯成指定的語言，然後將翻譯後的文字替換原來的字幕文字。最後，我們儲存翻譯後的字幕檔。

In this block of code, we're defining a function translate_srt_file_by_googletrans to translate an srt subtitle file into a specified language using Google translation service. We first create a translator object. Then, we open the srt subtitle file and translate each subtitle in it. We translate the text of each subtitle into the specified language and replace the original subtitle text with the translated text. Finally, we save the translated subtitle file.

In [ ]:
def translate_srt_file_by_googletrans(lang, sliced_part_srt, sliced_part_subtitle_srt):
  translator = Translator()  # We're creating a translator object.
  subtitles = pysrt.open(sliced_part_srt)  # We're opening the srt subtitle file.
  for subtitle in subtitles:  # For each subtitle in the subtitle file...
    translated_subtitle = translator.translate(text=subtitle.text, dest=lang)  # ...we translate its text into the specified language...
    subtitle.text = translated_subtitle.text  # ...and then we replace the original subtitle text with the translated text.
  subtitles.save(sliced_part_subtitle_srt, encoding='utf-8')  # We're saving the translated subtitle file.

在這段程式碼中，我們定義了一個函數 create_srt_files，用來創建 srt 字幕檔。首先，我們創建一個原始的字幕檔。然後，我們對給定的每一種語言創建一個對應的字幕檔。在每個字幕檔中，我們會將音訊檔的檔名和語言的名稱連接起來作為新的檔名。

In this block of code, we're defining a function create_srt_files to create srt subtitle files. We first create an original subtitle file. Then, we create a corresponding subtitle file for each given language. In each subtitle file, we concatenate the filename of the audio file and the name of the language to form a new filename.

In [ ]:
def create_srt_files(audio_filename, languages):
  # Create srt file for original caption
  with open(f'{audio_filename}.srt', 'w') as fp:  # We're creating an original subtitle file.
    pass
  # Create srt files for multilingual subtitles
  for language in languages:  # For each given language...
    with open(f'{audio_filename}_{language}.srt', "w") as fp:  # ...we create a corresponding subtitle file.
      pass

在這段程式碼中，我們定義了一個處理流程，以處理和翻譯 YouTube 影片的音訊和字幕。首先，我們創建了一個目錄來儲存音訊檔。然後，我們針對每個 YouTube 影片，我們下載其音訊檔，並將其轉換為 MP3 格式。接著，我們載入音訊檔，並使用 whisper 模型來進行轉寫。我們為每個音訊檔和每種語言創建一個對應的 srt 字幕檔。然後，我們將音訊檔切割成一分鐘的片段，並為每個片段進行轉寫。我們將轉寫結果寫入一個 srt 字幕檔，並將這個字幕檔與主字幕檔進行連接。我們將字幕翻譯成指定的語言，並將翻譯後的字幕檔與主字幕檔進行連接。最後，我們刪除所有切割過的音訊檔和字幕檔。

In this block of code, we're defining a workflow to process and translate the audio and subtitles of YouTube videos. We first create a directory to store the audio files. Then, for each YouTube video, we download its audio file and convert it to MP3 format. After that, we load the audio file and use the whisper model to transcribe it. We create a corresponding srt subtitle file for each audio file and each language. Then, we slice the audio file into one-minute chunks and transcribe each chunk. We write the transcription results into an srt subtitle file and concatenate this subtitle file with the main subtitle file. We translate the subtitles into the specified language and concatenate the translated subtitle file with the main subtitle file. Finally, we delete all the sliced audio files and subtitle files.

## The Main Program

In [ ]:
youtube_url = "https://www.youtube.com/watch?v="
# The key is the Youtube video's id
youtube_video = {
    "Diffusion_Model_原理1": "ifCDXFdeaaM"
}
languages = ['en', 'ja']

mp3_directory = './mp3/'
audio_directory = "./audio/"
if not os.path.isdir(mp3_directory):
    os.mkdir(mp3_directory)

for key, video_id in youtube_video.items():
  audio_filename = "audio_" + key
  download_Youtube_video_audio_file(youtube_url+video_id, audio_directory, audio_filename)
  audio_filepath = audio_directory + audio_filename
  mp3_filepath = mp3_directory + audio_filename + ".mp3"
  convert_audio_file_to_mp3_format(audio_filepath, mp3_filepath)
  mp3_file = AudioSegment.from_file(mp3_filepath, 'mp3')
  model = whisper.load_model("tiny")
  create_srt_files(audio_filename, languages)
  mp3_duration_minutes = int(mp3_file.duration_seconds // 60)
  seconds = round(mp3_file.duration_seconds - mp3_duration_minutes * 60, 2)

  for offset in range(mp3_duration_minutes + 1):
    filename = f"{audio_filename}_{offset}_{offset+1}.mp3"
    slice_audio(mp3_file, filename, offset)   
    result = model.transcribe(filename)
    output_directory = "./"
    srt_writer = get_writer("srt", output_directory)
    srt_writer(result, filename)
    main_srt = f'{audio_filename}.srt'
    sliced_part_srt = f"{audio_filename}_{offset}_{offset+1}.srt"
    concatenate_srt_file(main_srt, sliced_part_srt, offset)

    for language in languages:
      main_subtitle_srt = f'{audio_filename}_{language}.srt'
      sliced_part_subtitle_srt = f"{audio_filename}_{language}_{offset}_{offset+1}.srt"
      translate_srt_file_by_googletrans(language, sliced_part_srt, sliced_part_subtitle_srt)
      concatenate_srt_file(main_subtitle_srt, sliced_part_subtitle_srt, offset)
      os.remove(sliced_part_subtitle_srt)
    os.remove(filename)
    os.remove(sliced_part_srt)

Download url: https://www.youtube.com/watch?v=ifCDXFdeaaM
Please open https://www.google.com/device and input code LFPD-YQND
Press enter when you have completed this step.
Load the audio file
Convert the audio file to MP3 format



100%|█████████████████████████████████████| 72.1M/72.1M [00:01<00:00, 68.5MiB/s]


Duration: 13 minutes 55.76 seconds

Minute 0
Transcribe the chunked video
Translate the caption into english.
Translate the caption into japanese.
Done

Minute 1
Transcribe the chunked video
Translate the caption into english.
Translate the caption into japanese.
Done

Minute 2
Transcribe the chunked video
Translate the caption into english.
Translate the caption into japanese.
Done

Minute 3
Transcribe the chunked video
Translate the caption into english.
Translate the caption into japanese.
Done

Minute 4
Transcribe the chunked video
Translate the caption into english.
Translate the caption into japanese.
Done

Minute 5
Transcribe the chunked video
Translate the caption into english.
Translate the caption into japanese.
Done

Minute 6
Transcribe the chunked video
Translate the caption into english.
Translate the caption into japanese.
Done

Minute 7
Transcribe the chunked video
Translate the caption into english.
Translate the caption into japanese.
Done

Minute 8
Transcribe the chun